# Home Value Prediction

## Project Goals
* Identify key features that can be used to create an effective predictive model.
* Use regression models to make house price predictions.
* Use findings to make recommendations and establish a foundation for future work to improve model's performance.

## Imports

In [1]:
#list all of the imports I used in this notebook or in a module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

#import my personal modules created for this report
import wrangle as w
#import explore as e
#import model as m

## Acquire

* Data acquired from the Codeup mySQL server on November 16, 2022
* It contained 52,441 rows and 6 columns
* Each observation represents a single-family home
* Each column represents a feature of those homes

In [2]:
#import unclean data from Codeup mySQL server
zillow_unclean = w.acquire_zillow_data()

## Prepare

#### Dropped rows:
* Duplicates   
* Rows having 0 bedrooms AND 0 bathrooms 
* Rows having more than 10,000 square feet (because these are large and can scew the data)
* Rows containing null values in any column

#### Created features
* ```county``` (names based on the fips code):  
    - 6037: LA
    - 6059: Orange 
    - 6111: Ventura 
* ```bath_bed_ratio``` 
    - Column displaying bathrooms/bedrooms

#### Other prep
* Split data into train, validate, and test (65/20/15)

In [3]:
#split and prepare the data for exploration and modeling
train_exp, X_train, y_train, \
X_validate, y_validate, \
X_test, y_test = w.big_zillow_wrangle(zillow_unclean, 'home_value')

### Data dictionary
| Feature | Definition | Type |
|:--------|:-----------|:-------
| **bathroooms** | The number of bathrooms in the home. |
| bedrooms | The number of bedrooms in the home. |
|**???**| Definition| *type*|
|**???**| Definition| *type*|
|**???**| Definition| *type*|
|**???**| Definition| *type*|
|**???**| Definition| *type*|
|**Target variable**
|**home_value**| The tax-assessed value of the home. | *float* |

### A look at the data

In [4]:
#show the first few rows of the train data
train_exp.head()

,parcelid,bathrooms,bedrooms,home_value,square_feet,year_built,bath_bed_ratio,county
2626,11886205,2.0,3,833611.0,2454,1933,0.666667,LA
19606,12412300,3.0,5,435860.0,2274,1976,0.600000,LA
40531,13954406,1.5,3,490013.0,1628,1959,0.500000,Orange
30010,11667890,3.0,3,1791473.0,2106,1973,1.000000,LA
5478,12184817,2.0,4,304269.0,1482,1915,0.500000,LA


## Explore

### 1. What percentage of our customers have churned?

In [ ]:
#show percentage of customers who churned from the train data
e.churn_percentage(train)

**A little over 1/4 of customers in the training data churn.**

## 2. Does a customer's tenure affect churn? 

In [ ]:
#plot tenure of those who churned and didn't churn
e.tenure_viz(train)

**It looks like those who have been a customer a shorter amount of time churn more**

**In order to confirm this prediction, I will now do a T-test to test for a significant difference between tenure of those who churn and don't churn**
* My confidence interval is 95%
* Resulting alpha is .05

In [ ]:
#set alpha for my testing
alpha = 0.05

**To run a T-test we have to check our assumptions**
* **Independent:** Yes. Two separate groups churn and no_churn
* **Normal distribution:** Yes. Enough observations
* **Equal variances:** After running the ```stats.levene```, p < alpha so they are NOT equal and we must use ```equal_var = False```

**Set Hypothesis**

$H_0$ = There is no difference in tenure between those who churn and don't churn 

$H_a$ = There is a difference in tenure between those who churn and don't churn 

In [ ]:
#run t-test and get results
e.tenure_ttest(train)

**p < alpha, so we can reject the null hypothesis and say there is probably a difference in tenure between those who churn and don't churn.** 

***Based on the visualizations and the results from the T-test, I think that tenure should be used in my model to help with the model's accuracy.***

## 3. Do higher monthly charges affect churn?

In [ ]:
#plot monthly charges of those who churned and didn't churn
e.monthly_charges_viz(train)

**It looks like those who churn have higher monthly charges**

**In order to confirm this prediction, I will now do a T-test to test if average monthly charges of those who churn are higher than those who don't churn**
* My confidence interval is 95%
* Resulting alpha is .05

**To run a T-test we have to check our assumptions**
* **Independent:** Yes. Two separate groups churn and no_churn
* **Normal distribution:** Yes. Enough observations
* **Equal variances:** After running the ```stats.levene```, p < alpha so they are NOT equal and we must use ```equal_var = False```

$H_0$ = Average monthly charges of those who churn $\leq$ average monthly charges of those who don't churn

$H_a$ = Average monthly charges of those who churn $>$ average monthly charges of those who don't churn

In [ ]:
#run t-test and get results
e.monthly_charges_ttest(train)

**Since t > 0 and p/2 < alpha we can reject the null hypothesis and say monthly charges of those who churn are higher than those who don't churn.**

***Based on the visualizations and the results from the T-test, I think that monthly charges should be used in my model to help with the model's accuracy.***

## 4. Does contract type affect churn?

In [ ]:
#plot different contract types compared to churn
e.contract_type_viz(train)

**It looks like those who have a month-to-month contract churn well above the average churn rate. I predict there is a relationship between contract type and churn**

**In order to confirm this prediction, I will now do a $\chi^2$ test to check if there is a relationship between contract type and churn**
* My confidence interval is 95%
* Resulting alpha is .05

$H_0$ = Contract type and churn are not related.

$H_a$ = Contract type and churn are related.

In [ ]:
#run χ^2 test and get results
e.contract_type_chi(train)

**Based on the visualizations and the results from the $\chi^2$ test, I think that contract type should be used in my model to help with the model's accuracy.**

## 5. Do those with no tech support churn more?

In [ ]:
#plot tech support customers compared to churn
e.tech_support_viz(train)

**It looks like those who don't have tech suport might churn more. I predict there is a relationship between having tech support and churn**

$H_0$ = There is no relationship between having tech support and churn.

$H_a$ = There is a relationship between having tech support and churn.

In [ ]:
#run χ^2 test and get results
e.tech_support_chi(train)

**p < alpha, we can reject the null hypothesis and say there is evidence that tech support and churn are related.**

**Based on the visualizations and the results from the $\chi^2$ test, I think that tech support should be used in my model to help with the model's accuracy.**

### Exploration Summary
* Tenure seems to be a driver of churn
* Monthly_charges seem to be a driver of churn
* Contract_type seems to be a driver of churn
* Tech_support seems to be a driver of churn

### Features I'm moving on with
| Feature | Reason | 
|:--------|:-----------|
|tenure| From our t-test we saw there is probably a difference in tenure between those who churn and don't churn.|
|monthly_charges| From our t-test we saw there is probably a difference in monthly charges between those who churn and don't churn.|
|contract_type| From our $\chi^2$ test we saw there is probably a relationship between contract type and churn.|
|tech_support| From our $\chi^2$ test we saw there is probably a relationship between contract type and churn.|

In [ ]:
#save drivers into a list
drivers = ['churn_encoded', 'tenure', 'monthly_charges', 'contract_type', 'tech_support']

### Features I'm not moving on with

| Feature | Reason | 
|:--------|:-----------|
|gender| No evidence that gender is a driver of churn
|senior_citizen| No evidence that senior_citizen is a driver of churn
|partner| Remove noise
|dependents| Remove noise
|phone_service| No evidence that phone_service is a driver of churn
|multiple_lines| No evidence that multiple_lines is a driver of churn
|online_security| Remove noise
|online_backup| Remove noise
|device_protection| Remove noise
|streaming_tv| Remove noise
|streaming_movies| Remove noise
|paperless_billing| Remove noise
|total_charges| There is a strong correlation between total_charges and tenure and between total_charges and monthly_charges. Since I will take tenure and monthly_charges into my model, total_charges is not needed.
|internet_service_type| Remove noise
|payment_type| Remove noise


# Modeling

* I will use accuracy as my evaluation metric
* I had looked at recall, but we cannot determine that false negatives are more costly unless we get more information from the company.
* Didn't churn customers make up 73% of the train data

In [ ]:
#prepare data for modeling
X_train, y_train, X_validate, y_validate, X_test, y_test = m.prep_for_model(train, validate, test, "churn_encoded", drivers)

In [ ]:
#baseline based on the percentage of the mode
baseline = y_train.value_counts(normalize=True).max()
#print results
print(f'Baseline accuracy: {baseline:.2%}')

### Model Testing
* I evaluated on the four different model types and various hyperparameter configurations below
* Models were evaluated on train and validate data to prevent overfitting
* The model that performed the best was then run on the test data

### Decision Tree

In [ ]:
#get decision tree model results
m.decision_tree_results(X_train, y_train, X_validate, y_validate)

### Random Forest

In [ ]:
#get random forest model results
m.random_forest_results(X_train, y_train, X_validate, y_validate)

### KNN

In [ ]:
#get KNN model results
m.knn_results(X_train, y_train, X_validate, y_validate)

### Logistic Regression

In [ ]:
#get logistic regression model results
m.log_results(X_train, y_train, X_validate, y_validate)

### Comparing Models
* All models perfomed better than baseline for both train and validate data.
* KNN had a high train accuracy and lower validate accuracy so it is probably over fit.

### Best model: Random Forest Model 
* I chose the random forest model because the difference in accuracy between the train and validate is less than 1%
* I will now run the random forest model on test data to gauge how it will perform on future data

In [ ]:
#get best model results on test data
m.best_model(X_train, y_train, X_test, y_test)

**This model performs better than baseline and with very similar accuracy across all three data sets (train, validate and test).**

In [ ]:
#get best model results across all data
m.best_model_comparison(X_train, y_train, X_validate, y_validate, X_test, y_test)

## Conclusion

### Summary
* About 1/4 of our customers churn
* Tenure seems to be a driver of churn
* Monthly_charges seem to be a driver of churn
* Contract_type seems to be a driver of churn
* Tech_support seems to be a driver of churn


### Recommendations
* To reduce churn, I would encourage the company to think about taking the following actions:
    * Seek out young customers and give discounts (lower monthly cost) in order to keep customers longer (increase tenure)
    * Seek out month-to-month contract customers and encourage them to sign a longer contract maybe through discounts
    * Seek out those who don't have tech support and give a free trial of tech support.

### Next Steps
* In the next iteration:
    * Look into how customers with internet add-ons are the same people then can see if they churn more or less.
    * Look into cost of add-ons and see if there are bundles that might reduce monthly charges. That could help us entice more customers to stay.